In [1]:
## Initial setup
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
import os
from dotenv import load_dotenv

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN)
backend = service.backend(name="ibm_rensselaer")

In [ ]:
# Create circuit
qc = qiskit.QuantumCircuit(4)
qc.initialize('0000')

# transform input of 0s to superposition
qc.h(0)

qc.h(1)

qc.x(2)
qc.h(2)

qc.x(3)
qc.h(3)

# apply Uf
qc.x(0)

qc.ccx(0, 1, 3)

qc.x(0)
qc.x(1)

qc.ccx(0, 1, 3)

qc.cx(0, 1)

qc.ccx(1, 2, 3)

qc.cx(0, 1)

qc.x(1)

# finally measure
qc.measure_all()


qc.draw()

┌──────────────────────┐┌───┐┌───┐     ┌───┐                         »
   q_0: ┤0                     ├┤ H ├┤ X ├──■──┤ X ├──■────■─────────■───────»
        │                      │├───┤└───┘  │  ├───┤  │  ┌─┴─┐     ┌─┴─┐┌───┐»
   q_1: ┤1                     ├┤ H ├───────■──┤ X ├──■──┤ X ├──■──┤ X ├┤ X ├»
        │  Initialize(0,0,0,0) │├───┤┌───┐  │  └───┘  │  └───┘  │  └───┘└───┘»
   q_2: ┤2                     ├┤ X ├┤ H ├──┼─────────┼─────────■────────────»
        │                      │├───┤├───┤┌─┴─┐     ┌─┴─┐     ┌─┴─┐          »
   q_3: ┤3                     ├┤ X ├┤ H ├┤ X ├─────┤ X ├─────┤ X ├──────────»
        └──────────────────────┘└───┘└───┘└───┘     └───┘     └───┘          »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                                                             »
«         ░ ┌─┐         
«   q_0: ─░─┤M├─────────
«         ░ └╥┘┌─┐      
«   q_1: ─░──╫─┤M├──────
«         ░  ║ └╥┘┌─┐   
«   q_2: ─░──╫──╫─┤M├───
«         ░  ║  ║ └╥┘┌─┐
«   q_3: ─░──╫──╫──╫─┤M├
«         ░  ║  ║  ║ └╥┘
«meas: 4/════╩══╩══╩══╩═
«            0  1  2  3

In [7]:
# Transpile and execute
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
qc_transpiled = pm.run(qc)

job = backend.run(qc_transpiled, shots=10**4)
result = job.result()

/tmp/ipykernel_28547/3748150143.py:8: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(qc_transpiled, shots=10**4)


In [14]:
# Display results
counts = result.get_counts()
print("z x3 x2 x1 | frequency")
for i in range(16):
    binary = format(i, '04b')
    print(binary, "      |", counts.get(binary, 0))

z x3 x2 x1 | frequency
0000       | 991
0001       | 937
0010       | 766
0011       | 733
0100       | 940
0101       | 419
0110       | 198
0111       | 735
1000       | 343
1001       | 954
1010       | 408
1011       | 470
1100       | 821
1101       | 528
1110       | 196
1111       | 561


In [15]:
# Simulation comparison
from qiskit_aer import AerSimulator
simulator = AerSimulator()
pm = generate_preset_pass_manager(optimization_level=1, backend=simulator)
qc_transpiled = pm.run(qc)

simulation = simulator.run(qc_transpiled, shots=10**4)
simulation_counts = simulation.result().get_counts()

print("z x3 x2 x1 | frequency")
for i in range(16):
    binary = format(i, '04b')
    print(binary, "      |", simulation_counts.get(binary, 0))

z x3 x2 x1 | frequency
0000       | 634
0001       | 615
0010       | 632
0011       | 613
0100       | 635
0101       | 629
0110       | 640
0111       | 587
1000       | 607
1001       | 642
1010       | 649
1011       | 644
1100       | 623
1101       | 579
1110       | 636
1111       | 635
